# Cuaderno Clase PLN - Preprocesamiento Avanzado


# Instalaciones e Importaciones

In [1]:
# Instalar librerías (si no están ya en Colab)
!pip install nltk spacy > /dev/null
!python -m spacy download es_core_news_sm > /dev/null # Modelo pequeño de español para spaCy

In [2]:
# Importar librerías
import nltk
import spacy
import re # Para expresiones regulares (limpieza)
import string # Para signos de puntuación

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# Cargar modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')
print("Modelo de spaCy 'es_core_news_sm' cargado.")

Modelo de spaCy 'es_core_news_sm' cargado.


In [5]:
# Cargar stopwords en español de NLTK
stopwords_es = nltk.corpus.stopwords.words('spanish')

In [ ]:
# Ejemplo de texto
texto_ejemplo = "Los niños corrían rápidamente por el parque, jugando y riendo. ¡Qué día más lindo!"

# 7. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:**

Dado el siguiente conjunto de frases (reviews de películas):
1.  Definir una función `preprocesar_nltk(texto)` que:
    *   Limpie el texto (minúsculas, números, puntuación).
    *   Tokenice.
    *   Quite stopwords (usando la lista de NLTK).
    *   Aplique Stemming (con `SnowballStemmer`).
    *   Devuelva la lista de stems.
2.  Definir una función `preprocesar_spacy(texto)` que:
    *   Limpie el texto (minúsculas, números, puntuación - cuidado con no quitar espacios necesarios para spaCy).
    *   Procese el texto con `nlp()`.
    *   Devuelva la lista de lemas de los tokens que no sean stopwords (`token.is_stop`) y sean alfabéticos (`token.is_alpha`).
3.  Aplicar ambas funciones a cada frase del dataset `reviews`.
4.  Imprimir los resultados de ambas funciones para cada frase, uno debajo del otro, para poder comparar.
5.  **Observar:** ¿Qué diferencias notables encuentran? ¿En qué casos un método parece funcionar "mejor" que el otro?

In [25]:
# Dataset para el ejercicio
reviews = [
    "Una película emocionante con actuaciones brillantes. ¡Me encantó!",
    "Muy aburrida y lenta. El guión era predecible y los actores no convencían.",
    "Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.",
    "¡Qué gran comedia! Me reí sin parar durante toda la película.",
    "Un documental necesario que aborda temas importantes con profundidad y sensibilidad."
]

In [21]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

def preprocesar_nltk(texto):
    # 1. Minúsculas
    texto = texto.lower()
    # 2. Quitar números (usando expresiones regulares)
    texto = re.sub(r'\d+', '', texto)
    # 3. Quitar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation + '¡¿'))
    # 4. Quitar espacios extra
    texto = texto.strip()
    # 5. Tokenizar
    tokens = nltk.word_tokenize(texto, language='spanish')
    # 6. Quitar stopwords
    tokens_limpios = [palabra for palabra in tokens if palabra not in stopwords_es]

    stems_nltk = [stemmer.stem(token) for token in tokens_limpios]

    return "Stems resultantes (NLTK): ", stems_nltk

In [24]:
def preprocesar_spacy(texto):
  # 1. Minúsculas
  texto = texto.lower()
  # 2. Quitar números
  texto = re.sub(r'\d+', '', texto)
  # 3. Quitar puntuación (dejamos espacios)
  texto = texto.translate(str.maketrans(string.punctuation + '¡¿', ' ' * len(string.punctuation + '¡¿')))
  # 4. Quitar espacios extra
  texto = re.sub(r'\s+', ' ', texto).strip()

  doc = nlp(texto)
  lemas_spacy = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

  return "Lemas resultantes (spaCy): ", lemas_spacy

In [22]:
stems_nltk = [preprocesar_nltk(review) for review in reviews]
stems_spacy = [preprocesar_spacy(review) for review in reviews]

print(stems_nltk)
print(stems_spacy)

[('Stems resultantes (NLTK): ', ['pelicul', 'emocion', 'actuacion', 'brillant', 'encant']), ('Stems resultantes (NLTK): ', ['aburr', 'lent', 'guion', 'predec', 'actor', 'convenc']), ('Stems resultantes (NLTK): ', ['efect', 'especial', 'impresion', 'histori', 'dej', 'des']), ('Stems resultantes (NLTK): ', ['gran', 'comedi', 'rei', 'par', 'tod', 'pelicul']), ('Stems resultantes (NLTK): ', ['documental', 'necesari', 'abord', 'tem', 'import', 'profund', 'sensibil'])]
[('Lemas resultantes (spaCy): ', ['película', 'emocionante', 'actuación', 'brillante', 'encantar']), ('Lemas resultantes (spaCy): ', ['aburrido', 'lento', 'guión', 'predecible', 'actor', 'convencer']), ('Lemas resultantes (spaCy): ', ['efecto', 'especial', 'impresionante', 'historia', 'dejar', 'desear']), ('Lemas resultantes (spaCy): ', ['comedia', 'reí', 'parar', 'película']), ('Lemas resultantes (spaCy): ', ['documental', 'necesario', 'abordar', 'tema', 'importante', 'profundidad', 'sensibilidad'])]


Conclusion: NLTK tiende a cortar las palabras mientras que Spacy interpreta prefectamente.